In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set default to run on the GPU if available (for the speed up)
if torch.cuda.is_available():
    torch.cuda.set_device(device)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [5]:
from torchtext import data, datasets
# set up fields
TEXT = data.Field(lower=True, include_lengths=True, batch_first=True, fix_length=100)
LABEL = data.Field(sequential=False)

# make splits for data
train, test = datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train, max_size=1000)
LABEL.build_vocab(train)

In [6]:
train_iter, test_iter = data.BucketIterator.splits(
    (train, test), batch_size=64, device=0)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [7]:
print(len(train_iter))

391


In [8]:
import torch.nn as nn 
from torch.nn import functional as F
n_hidden = 128
num_words  = len(TEXT.vocab)

class LSTMModel(nn.Module):

    def __init__(self, num_words, n_hidden):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(num_words, n_hidden)
        self.LSTM = nn.LSTM(n_hidden, n_hidden, batch_first=True,)
        self.fc = nn.Linear(n_hidden, 1)

    def forward(self, review):
        
        x = self.embedding(review)
        
        rnn_out, _ = self.LSTM(x)
     
        fc_out = self.fc(rnn_out[:, -1 ]) # Only need the last output of the rnn
        
        return fc_out
        
    
model = LSTMModel(num_words + 2, n_hidden)
model.to(device)

LSTMModel(
  (embedding): Embedding(1004, 128)
  (LSTM): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [9]:
loss_history = []
test_history = []
acc_history = []
n_epochs = 5

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters() , 1e-3)
for epoch in range(1, n_epochs + 1):
    epoch_total_loss = 0
    for i, batch in enumerate(train_iter):
        optimizer.zero_grad() # Clears existing gradients from previous epoch
        
        output = model(batch.text[0])

        
        
        loss = criterion(output, batch.label.float().view(-1, 1) - 1)
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly

        epoch_total_loss += loss.item() # Keep track of the total loss
    loss_history.append(epoch_total_loss/len(train_iter))

        
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(epoch_total_loss/ len(train_iter)))

Epoch: 1/5............. Loss: 0.6844
Epoch: 2/5............. Loss: 0.6131
Epoch: 3/5............. Loss: 0.5108
Epoch: 4/5............. Loss: 0.4554
Epoch: 5/5............. Loss: 0.4146
